### Automatic Values

Enumerations have a builtin mechanism to auto assign values to members.

This is often useful when you migth have a simple associated integer value that is sequential, for example `1, 2, 3, 4, ...`

We can easily let enums assign their own values this way, using the `auto()` function in the enum module.

By default it will use sequential integers, starting at `1`:

In [1]:
import enum

In [2]:
class State(enum.Enum):
    WAITING = enum.auto()
    STARTED = enum.auto()
    FINISHED = enum.auto()

In [3]:
for member in State:
    print(member.name, member.value)

WAITING 1
STARTED 2
FINISHED 3


We can actually mix in our own values too, but we have to be really careful - nothing in the Python documentation states what will/will not work - their only advice is ```Care must be taken if you mix auto with other values```. That's not saying much, and so I **never** mix auto-generated values and my own - just to be on the safe side.

This seems to work fine:

In [4]:
class State(enum.Enum):
    WAITING = 5
    STARTED = enum.auto()
    FINISHED = enum.auto()

In [5]:
for member in State:
    print(member.name, member.value)

WAITING 5
STARTED 6
FINISHED 7


But observe what happens here:

In [6]:
class State(enum.Enum):
    WAITING = enum.auto()
    STARTED = 1
    FINISHED = enum.auto()
    
for member in State:
    print(member.name, member.value)
    
State.__members__

WAITING 1
FINISHED 2


mappingproxy({'WAITING': <State.WAITING: 1>,
              'STARTED': <State.WAITING: 1>,
              'FINISHED': <State.FINISHED: 2>})

As you can see, `STARTED` ended up being an alias for `WAITING` - not what my intention was.

Using `@unique` does not solve the issue, although it does make it immediately clear that there is a problem:

In [7]:
try:
    @enum.unique
    class State(enum.Enum):
        WAITING = enum.auto()
        STARTED = 1
        FINISHED = enum.auto()
except ValueError as ex:
    print(ex)

duplicate values found in <enum 'State'>: STARTED -> WAITING


Enum classes use the `_generate_next_value_` method to generate these automatic values, and we can actually override this to provide our implementation of an automatic value. The default implemtation currently generates a sequence of numbers, but the actual algorithm is an implementation detail - i.e. we cannot rely on any specific sequence of values being generated.

We can however override it if we wish:

In [8]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        print(name, start, count, last_values)
        return 100
    
    a = enum.auto()
    b = enum.auto()
    c = enum.auto()

a 1 0 []
b 1 1 [100]
c 1 2 [100, 100]


As we can see the `last_values` property is a list of all the preceding values used for member. The `count` property is simply the number of enum members already created (including aliases!). The `name` property is the name of the member. The `start` argument is actually only used when we create enumerations using a functional approach (very similar to how we created named tuples) - but I am not going to cover this in this course (feel free to explore the Python docs, it's quite straightforward).

Let's see a more interesting example of how we could use this override. Let's say we want the associated values to be random integers, where we do not want duplicates.

In [9]:
import random

random.seed(0)

class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        while True:
            new_value = random.randint(1, 100)
            if new_value not in last_values:
                return new_value
            
    a = enum.auto()
    b = enum.auto()
    c = enum.auto()

In [10]:
for member in State:
    print(member.name, member.value)

a 50
b 98
c 54


Another example, shown in the Python docs is using the string of the member name as the value. In this example I choose to title case the name:

In [11]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        return name.title()  
    
    WAITING = enum.auto()
    STARTED = enum.auto()
    FINISHED = enum.auto()
    
for member in State:
    print(member.name, member.value)

WAITING Waiting
STARTED Started
FINISHED Finished


If we want to make our `_generate_next_value_` implementation reusable across more than one enumeration, we could create an enumeration that only implements this functionality, and then use that as the parent class to our other enumerations:

In [12]:
class NameAsString(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        return name.lower()

In [13]:
class Enum1(NameAsString):
    A = enum.auto()
    B = enum.auto()
    
class Enum2(NameAsString):
    WAIT = enum.auto()
    RUNNING = enum.auto()
    FINISHED = enum.auto()

In [14]:
for member in Enum1:
    print(member.name, member.value)
    
for member in Enum2:
    print(member.name, member.value)

A a
B b
WAIT wait
RUNNING running
FINISHED finished


### Note

Sometimes, we don't actually care about the associated value for each member. In that case we can certainly use `auto()`, but the problem might be that users of our enumeration rely on that associated value.

Later, if we want to add items to the enumeration (somewhere in the middle), our users' code would break.

We might therefore want to discourage our users from ever using the associated value, and only using the keys.

Although we can (and should) document this, we can also enforce this using a simple trick. We assign an instance of `object` as the value for each member. There is very little our users can then do with that value, and so we are ensuring their safety.

In [15]:
class State(enum.Enum):
    WAIT = object()
    RUNNING = object()
    FINISHED = object()

In [16]:
State.WAIT, State.RUNNING, State.FINISHED

(<State.WAIT: <object object at 0x7fab7807bd30>>,
 <State.RUNNING: <object object at 0x7fab7807bd40>>,
 <State.FINISHED: <object object at 0x7fab7807bda0>>)

In order for a user to use the value, they would have to first get a handle to the object instance itself - they would never get that back from a literal string, integer, etc.

Now, instead of remembering to use `object()` for every member, we could use a base class to make it reusable (and a consistent implementation), and the auto functionality:

In [17]:
class ValuelessEnum(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        return object()
    
class State(ValuelessEnum):
    WAIT = enum.auto()
    RUNNING = enum.auto()
    FINISHED = enum.auto()
    
class Errors(ValuelessEnum):
    NumberError = enum.auto()
    IndexError = enum.auto()
    TimeoutError = enum.auto()

In [18]:
State.WAIT, Errors.TimeoutError

(<State.WAIT: <object object at 0x7fab7807bdd0>>,
 <Errors.TimeoutError: <object object at 0x7fab7807be40>>)

By using a base class, we could technically change our implementation of how the values are generated without having to touch our subclassed enumerations:

In [19]:
class ValuelessEnum(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        while True:
            new_value = random.randint(1, 100)
            if new_value not in last_values:
                return new_value
    
class State(ValuelessEnum):
    WAIT = enum.auto()
    RUNNING = enum.auto()
    FINISHED = enum.auto()
    
class Errors(ValuelessEnum):
    NumberError = enum.auto()
    IndexError = enum.auto()
    TimeoutError = enum.auto()

In [20]:
State.WAIT, Errors.TimeoutError

(<State.WAIT: 6>, <Errors.TimeoutError: 39>)

### Auto and Aliases

I want to touch back on the `count` argument of `_generate_next_value_` when are are dealing with aliases.

Since the default implementation of `_generate_next_value_` generates sequential integer numbers, we can never create aliases using this default.

However, nothing stops us from doing so when we have our own implementation of that function. In that case `count` will reflect the number of items created, **including** any aliases.

In [21]:
class Aliased(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        print(f'count={count}')
        if count % 2 == 1:
            # odd, make this member an alias of the previous one
            return last_values[-1]
        else:
            # make a new value
            return last_values[-1] + 1
       
    GREEN = 1
    GREEN_ALIAS = 1
    RED = 10
    CRIMSON = enum.auto()
    BLUE = enum.auto()
    AQUA = enum.auto()

count=3
count=4
count=5


As you can see `_generate_next_value_` was called for the last three members of our enum, and reflect the number of items that were created to that point, including aliases.

In [22]:
list(Aliased)

[<Aliased.GREEN: 1>, <Aliased.RED: 10>, <Aliased.BLUE: 11>]

In [23]:
Aliased.__members__

mappingproxy({'GREEN': <Aliased.GREEN: 1>,
              'GREEN_ALIAS': <Aliased.GREEN: 1>,
              'RED': <Aliased.RED: 10>,
              'CRIMSON': <Aliased.RED: 10>,
              'BLUE': <Aliased.BLUE: 11>,
              'AQUA': <Aliased.BLUE: 11>})